# DeepLabCut 2.2 Toolbox on 2 mice data
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628250004229-KVYD7JJVHYEFDJ32L9VJ/DLClogo2021.jpg?format=1000w)

https://github.com/DeepLabCut/DeepLabCut

### This notebook illustrates how to use COLAB for a multi-animal DeepLabCut (maDLC) Demo 3 mouse project:
- load our mini-demo data that includes a pretrained model and unlabeled video.
- analyze a novel video.
- assemble animals and tracklets.
- create quality check plots and video.

### To create a full maDLC pipeline please see our full docs: https://deeplabcut.github.io/DeepLabCut/README.html
- Of interest is a full how-to for maDLC: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html
- a quick guide to maDLC: https://deeplabcut.github.io/DeepLabCut/docs/tutorial.html
- a demo COLAB for how to use maDLC on your own data: https://github.com/DeepLabCut/DeepLabCut/blob/main/examples/COLAB/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb

### To get started, please go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
# Install the latest DeepLabCut version:
!pip install "deeplabcut[tf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 42.5 M

No information needs edited in the cells below, you can simply click run on each:

### Download our Demo Project from our server:

In [3]:
# Download our demo project:
import requests
from io import BytesIO
from zipfile import ZipFile

url_record = 'https://zenodo.org/api/records/7883589'
response = requests.get(url_record)
if response.status_code == 200:
    file = response.json()['files'][0]
    title = file['key']
    print(f"Downloading {title}...")
    with requests.get(file['links']['self'], stream=True) as r:
        with ZipFile(BytesIO(r.content)) as zf:
            zf.extractall(path='/content')
else:
    raise ValueError(f'The URL {url_record} could not be reached.')

## Analyze a novel 2 mouse video with our maDLC DLCRNet, pretrained on 3 mice data (i.e., here you extract detections and association costs):

In [ ]:
import deeplabcut as dlc
import os
import requests

def descargar_y_guardar_video(url, destino):
    respuesta = requests.get(url)
    with open(destino, 'wb') as archivo:
        archivo.write(respuesta.content)

opcion = 5 # Cambiar según la opción deseada, 1, 2,3

if opcion == 1:
    url_record = 'https://firebasestorage.googleapis.com/v0/b/socialx-a8a3e.appspot.com/o/Videos%2FTrial_2.mp4?alt=media&token=72288843-07d2-45d5-89b6-9d009b57d746'
    name = 'Trial_2.mp4'
elif opcion == 2:
    url_record = 'https://firebasestorage.googleapis.com/v0/b/socialx-a8a3e.appspot.com/o/Videos%2FR4_HFH.mp4?alt=media&token=b29a5619-90ef-4d87-84a3-2c0af9b6ebc4'
    name = 'R4_HFH.mp4'
elif opcion == 3:
    url_record = 'https://firebasestorage.googleapis.com/v0/b/socialx-a8a3e.appspot.com/o/Videos%2Fvideocompressed1.mp4?alt=media&token=2b59391c-b3f2-4d7a-bdeb-ed846519b0bd'
    name = 'videocompressed1.mp4'

elif opcion == 4:
    url_record = 'https://firebasestorage.googleapis.com/v0/b/socialx-a8a3e.appspot.com/o/Videos%2FTrial_2_medio.mp4?alt=media&token=5a1ae9e9-60f3-4785-9e04-8ebf9a0576ff'
    name = 'Trial_2_medio.mp4'

elif opcion == 5:
  url_record= 'https://firebasestorage.googleapis.com/v0/b/socialx-a8a3e.appspot.com/o/Videos%2FR4_HFH_Oscuro.avi?alt=media&token=044a5508-b9ad-4ab6-bdfc-6bc572dc4938'
  name='R4_HFH_Oscuro.avi'

else:
    url_record = 'No existe'
    name = 'No existe'
    # Terminar el proceso
    print( 'Video y direccion no existe',name, url_record )
    exit()


project_path = "/content/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")

# Crear el directorio si no existe
video_path = os.path.join(project_path, "videos")
if not os.path.exists(video_path):
    os.makedirs(video_path)

video = os.path.join(video_path, name)

# Descargar y guardar el video
descargar_y_guardar_video(url_record, video)

# Determinar el tipo de video basado en la extensión del archivo
video_type = 'mp4' if name.endswith('.mp4') else 'avi'

# Realizar análisis de video con DeepLabCut
dlc.analyze_videos(config_path, [video], shuffle=0, videotype=video_type, auto_track=False)


Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Starting to analyze %  /content/demo-me-2021-07-14/videos/R4_HFH_Oscuro.avi
Loading  /content/demo-me-2021-07-14/videos/R4_HFH_Oscuro.avi
Duration of video [s]:  925.5 , recorded with  30.0 fps!
Overall # of frames:  27765  found with (before cropping) frame dimensions:  640 240
Starting to extract posture from the video(s) with batchsize: 8


 13%|█▎        | 3560/27765 [15:53<1:43:47,  3.89it/s]

### Next, you compute the local, spatio-temporal grouping and track body part assemblies frame-by-frame:

In [ ]:
TRACK_METHOD = "ellipse"  # Could also be "box", but "ellipse" was found to be more robust on this dataset.

dlc.convert_detections2tracklets(
    config_path,
    [video],
    videotype=video_type,
    shuffle=0,
    track_method=TRACK_METHOD,
    ignore_bodyparts=["tail1", "tail2", "tailend"],  # Some body parts can optionally be ignored during tracking for better assembly (but they are used later)
)

Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0
Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Analyzing /content/demo-me-2021-07-14/videos/videocompressed1DLC_dlcrnetms5_demoJul14shuffle0_20000.h5


100%|██████████| 2330/2330 [00:01<00:00, 1266.52it/s]
2330it [00:06, 354.46it/s] 

The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.


### Reconstruct full animal trajectories (tracks from tracklets):

In [ ]:
dlc.stitch_tracklets(
    config_path,
    [video],
    videotype=video_type,
    shuffle=0,
    track_method=TRACK_METHOD,
    n_tracks=3,
)

Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4


100%|██████████| 3/3 [00:00<00:00, 31068.92it/s]


## Create a pretty video output:

In [ ]:
#Filter the predictions to remove small jitter, if desired:
dlc.filterpredictions(config_path,
                                 [video],
                                 shuffle=0,
                                 videotype=video_type,
                                 track_method = TRACK_METHOD)
dlc.create_labeled_video(
    config_path,
    [video],
    videotype=video_type,
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
    track_method=TRACK_METHOD,
)


Filtering with median model /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Saving filtered csv poses!
Starting to process video: /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:29<00:00, 79.15it/s]


[True]

Now, on the left panel if you click the folder icon, you will see the project folder "demo-me.."; click on this and go into "videos" and you can find the "..._id_labeled.mp4" video, which you can double-click on to download and inspect!

### Create Plots of your data:

> after running, you can look in "videos", "plot-poses" to check out the trajectories! (sometimes you need to click the folder refresh icon to see it). Within the folder, for example, see plotmus1.png to vide the bodyparts over time vs. pixel position.



In [ ]:
dlc.plot_trajectories(config_path, [video], shuffle=0,videotype=video_type, track_method=TRACK_METHOD)

Loading  /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory
